# NOTE: This is a Workin In Progress notebook - Uncomplete and may not work as expected.

In [1]:
import tensorflow as tf
from keras.layers import Input, Conv2D, Concatenate, BatchNormalization, ReLU, AveragePooling2D, GlobalAveragePooling2D, Dense
import datetime

2025-07-01 06:47:15.112075: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def dense_block(x, n_layers, n_filters):
    for l in range(n_layers):
        out = BatchNormalization()(x)
        out = ReLU()(out)
        out = Conv2D(n_filters, kernel_size=3, padding='same')(out)
        x = Concatenate()([x, out])  # Dense connectivity
    return x

In [3]:
_dense_block = tf.keras.Model(
    inputs=Input(shape=(112, 112, 1)), 
    outputs=dense_block(Input(shape=(112, 112, 1)), n_layers=1, n_filters=32)
)
_dense_block.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │          4 │ input_layer_1[0]… │
│ (BatchNormalizatio… │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │        320 │ re_lu[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 112, 112,  │          0 │ input_layer_1[0]… │
│ (Concatenate)       │ 33)               │            │ conv2d[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 324 (1.27 KB)

 Trainable params: 322 (1.26 KB)

 Non-trainable params: 2 (8.00 B)

In [4]:
_dense_block.output_shape

(None, 112, 112, 33)

In [5]:
def transition_layer(x, reduction):
    x = BatchNormalization()(x)
    x = Conv2D(int(tf.keras.backend.int_shape(x)[-1] * reduction), kernel_size=1, padding='same')(x)
    x = AveragePooling2D(pool_size=2, strides=2)(x)
    return x

In [6]:
_transition_layer = tf.keras.Model(
    inputs=Input(shape=(112, 112, 32)), 
    outputs=transition_layer(Input(shape=(112, 112, 32)), reduction=0.5)
)
_transition_layer.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        128 │ input_layer_3[0]… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 112, 112,  │        528 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d   │ (None, 56, 56,    │          0 │ conv2d_1[0][0]    │
│ (AveragePooling2D)  │ 16)               │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 656 (2.56 KB)

 Trainable params: 592 (2.31 KB)

 Non-trainable params: 64 (256.00 B)

In [13]:
# Input layer
inputs = Input(shape=(32, 32, 3))

# Initial convolution
x = Conv2D(16, kernel_size=3, padding='same')(inputs)

# Dense Block 1
x = dense_block(x, n_layers=4, n_filters=12)
x = transition_layer(x, reduction=0.5)

# Dense Block 2
x = dense_block(x, n_layers=4, n_filters=12)
x = transition_layer(x, reduction=0.5)

# Classification head
x = BatchNormalization()(x)
x = ReLU()(x)
x = GlobalAveragePooling2D()(x)
outputs = Dense(10, activation='softmax')(x)  # For CIFAR-10 or similar

In [14]:
# Build the model
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 32, 32,    │        448 │ input_layer_5[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │      1,740 │ re_lu_1[0][0]     │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 32, 32,    │          0 │ conv2d_3[0][0],   │
│ (Concatenate)       │ 28)               │            │ conv2d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        112 │ concatenate_1[0]… │
│ (BatchNormalizatio… │ 28)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 28)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │      3,036 │ re_lu_2[0][0]     │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 32, 32,    │          0 │ concatenate_1[0]… │
│ (Concatenate)       │ 40)               │            │ conv2d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        160 │ concatenate_2[0]… │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │      4,332 │ re_lu_3[0][0]     │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 32, 32,    │          0 │ concatenate_2[0]… │
│ (Concatenate)       │ 52)               │            │ conv2d_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        208 │ concatenate_3[0]… │
│ (BatchNormalizatio… │ 52)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 52)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 32, 32,    │      5,628 │ re_lu_4[0][0]   

 Total params: 44,642 (174.38 KB)

 Trainable params: 43,602 (170.32 KB)

 Non-trainable params: 1,040 (4.06 KB)

In [8]:
log_dir = "./Data/DenseNet/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
log_dir

'./Data/DenseNet/20250701-064748'

In [28]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [31]:
x = tf.random.normal(shape=(1, 32, 32, 3))
y = tf.random.uniform(shape=(1, 10), maxval=10, dtype=tf.int32)
x.shape, y.shape

(TensorShape([1, 32, 32, 3]), TensorShape([1, 10]))

In [32]:
model.fit(
    x, y,
    epochs=1,
    steps_per_epoch=1,
    callbacks=[tensorboard_callback]
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.0000e+00 - loss: 138.4359


In [34]:
%%bash
/opt/conda/envs/py39_tensorflow/bin/tensorboard --logdir ./Data/DenseNet --host 0.0.0 --port 6006 

2025-07-01 06:56:26.293598: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.18.0 at http://0.0.0:6006/ (Press CTRL+C to quit)


Error while terminating subprocess (pid=2404816): 
